In [ ]:
#pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib pdfplumber pandas

In [ ]:
# --- Google Drive API imports ---
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow

In [11]:
import io
import re
import calendar
from datetime import datetime
from typing import List, Dict

import pandas as pd
import pdfplumber

# --- Google Drive API imports ---
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow

# ==============================
# Configuration
# ==============================
DRIVE_FOLDER_ID = "1NCK8WwAGGSpKq0cKXCXGm9YxEuj6GKOU"  # put your Google Drive folder ID
INCLUDE_SUBFOLDERS = True                      # set False to only scan the top folder
CSV_OUT = "data/electricity_fasce_monthly.csv"

# OAuth scope: read-only is enough
SCOPES = ["https://www.googleapis.com/auth/drive.readonly"]

# ==============================
# Parsing helpers 
# ==============================
MONTH_MAP = {
    "Gen": 1, "Feb": 2, "Mar": 3, "Apr": 4, "Mag": 5, "Giu": 6,
    "Lug": 7, "Ago": 8, "Set": 9, "Ott": 10, "Nov": 11, "Dic": 12,
}
ROW_START = re.compile(r'^(Gen|Feb|Mar|Apr|Mag|Giu|Lug|Ago|Set|Ott|Nov|Dic)-(\d{4})\b')
NUM_RE = re.compile(r'\d{1,3}(?:\.\d{3})*(?:,\d+)?|\d+(?:,\d+)?')

def itnum_to_float(s: str) -> float:
    """Convert Italian-formatted number to float."""
    s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except ValueError:
        return float("nan")

def extract_consumi_from_line(line: str):
    """Extract F1, F2, F3 kWh from a monthly row line."""
    line = re.sub(r'\s+', ' ', line.strip())
    line_wo_period = ROW_START.sub("", line, count=1).strip()
    nums = NUM_RE.findall(line_wo_period)
    vals = [itnum_to_float(n) for n in nums]
    if len(vals) < 3:
        return None
    return vals[0], vals[1], vals[2]

def parse_invoice_bytes(pdf_bytes: bytes, source_name: str) -> List[Dict]:
    """
    Parse the 'CONSUMO STORICO ENERGIA' monthly lines from a PDF (bytes).
    Returns a list of dict rows.
    """
    rows = []
    with pdfplumber.open(io.BytesIO(pdf_bytes)) as pdf:
        lines = []
        for page in pdf.pages:
            text = page.extract_text() or ""
            lines.extend(text.splitlines())

    for line in lines:
        m = ROW_START.match(line.strip())
        if not m:
            continue

        mon_it, year = m.group(1), int(m.group(2))
        month = MONTH_MAP.get(mon_it)
        if not month:
            continue

        date = datetime(year, month, 1).date()
        parsed = extract_consumi_from_line(line)
        if not parsed:
            continue

        F1_kWh, F2_kWh, F3_kWh = parsed
        days_in_month = calendar.monthrange(year, month)[1]

        rows.append({
            "date": date.isoformat(),
            "source_file": source_name,
            "days_in_month": days_in_month,
            "F1_kWh": F1_kWh,
            "F2_kWh": F2_kWh,
            "F3_kWh": F3_kWh,
            "total_kWh": F1_kWh + F2_kWh + F3_kWh,
            "F1_daily_kWh": F1_kWh / days_in_month,
            "F2_daily_kWh": F2_kWh / days_in_month,
            "F3_daily_kWh": F3_kWh / days_in_month,
        })
    return rows

# ==============================
# Google Drive helpers
# ==============================
def get_drive_service():
    """
    Authenticate and return a Drive API service.
    On first run, opens a browser to complete OAuth; stores token.json.
    """
    creds = None
    try:
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    except Exception:
        pass

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            # creds.refresh(Request())  # not needed with installed app flow if token valid
            pass
        else:
            flow = InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES)
            creds = flow.run_local_server(port=0)
        with open("token.json", "w") as token:
            token.write(creds.to_json())

    return build("drive", "v3", credentials=creds)

def list_folder_pdfs(service, folder_id: str, include_subfolders=True) -> List[Dict]:
    """
    Return list of dicts: {id, name, mimeType} for PDFs (and recurse if include_subfolders).
    """
    results = []

    def _list_in(folder):
        page_token = None
        while True:
            q = f"'{folder}' in parents and trashed=false"
            resp = service.files().list(
                q=q,
                fields="nextPageToken, files(id, name, mimeType)",
                pageToken=page_token,
                includeItemsFromAllDrives=True, supportsAllDrives=True
            ).execute()
            for f in resp.get("files", []):
                if f["mimeType"] == "application/pdf":
                    results.append(f)
                elif include_subfolders and f["mimeType"] == "application/vnd.google-apps.folder":
                    _list_in(f["id"])
            page_token = resp.get("nextPageToken")
            if not page_token:
                break

    _list_in(folder_id)
    return results

def download_file_bytes(service, file_id: str) -> bytes:
    """Download a Drive file (by id) into bytes."""
    request = service.files().get_media(fileId=file_id)
    fh = io.BytesIO()
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while not done:
        status, done = downloader.next_chunk()
        # You can print progress if desired: print(f"Downloaded {int(status.progress()*100)}%")
    return fh.getvalue()

# ==============================
# Main
# ==============================
def main():
    service = get_drive_service()

    print("Listing PDFs in Drive folder...")
    files = list_folder_pdfs(service, DRIVE_FOLDER_ID, INCLUDE_SUBFOLDERS)
    if not files:
        print("No PDFs found in the specified Drive folder.")
        return

    print(f"Found {len(files)} PDF(s). Parsing…")
    all_rows: List[Dict] = []
    for f in files:
        print('Doing', f)
        try:
            content = download_file_bytes(service, f["id"])
            rows = parse_invoice_bytes(content, f["name"])
            if rows:
                all_rows.extend(rows)
            else:
                print(f"[WARN] No monthly rows parsed from: {f['name']}")
        except Exception as e:
            print(f"[WARN] Failed to parse {f['name']}: {e}")

    df = pd.DataFrame(all_rows)
    if df.empty:
        print("⚠️ No consumption rows found across all PDFs.")
        return

    # If duplicates for the same month exist, keep the last by filename sort
    df.sort_values(["date", "source_file"], inplace=True)
    df = df.drop_duplicates(subset=["date"], keep="last")

    df.to_csv(CSV_OUT, index=False)
    print(f"✅ Saved CSV -> {CSV_OUT}")
    print(df.head(12).to_string(index=False))

if __name__ == "__main__":
    main()


Listing PDFs in Drive folder...
Found 111 PDF(s). Parsing…
Doing {'id': '1hQezXrsMLLX1obGHLjfYD0z9LuC3OyiM', 'name': 'DETT_1_E_0157095_20201229.pdf', 'mimeType': 'application/pdf'}
[WARN] No monthly rows parsed from: DETT_1_E_0157095_20201229.pdf
Doing {'id': '1xrjwyiRmZ4dLdR_1RTupVpC5z_k5_lWy', 'name': 'DETT_1_E_0142928_20201130.pdf', 'mimeType': 'application/pdf'}
[WARN] No monthly rows parsed from: DETT_1_E_0142928_20201130.pdf
Doing {'id': '1rGmILMa9ShGXMWwTed8UqWuWjKbJybYs', 'name': 'DETT_1_E_0127356_20201029.pdf', 'mimeType': 'application/pdf'}
[WARN] No monthly rows parsed from: DETT_1_E_0127356_20201029.pdf
Doing {'id': '1qJl52F1jiJp8WdebABnFDAcfoypDJGrO', 'name': 'DETT_1_E_0115336_20200930.pdf', 'mimeType': 'application/pdf'}
[WARN] No monthly rows parsed from: DETT_1_E_0115336_20200930.pdf
Doing {'id': '1R8c4Evy0NEm9ovLOqUxSKGby3PSxJS4H', 'name': 'DETT_1_E_0103231_20200828.pdf', 'mimeType': 'application/pdf'}
[WARN] No monthly rows parsed from: DETT_1_E_0103231_20200828.pdf
D

In [1]:
import io
import os
import re
import calendar
from datetime import datetime
from typing import List, Dict, Tuple

import pandas as pd
import pdfplumber

# --- Google Drive API imports ---
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

# ==============================
# Configuration
# ==============================
FOLDER_IDS = [
    "1NCK8WwAGGSpKq0cKXCXGm9YxEuj6GKOU",   # <-- replace with real Drive folder IDs (or URLs; see folder_id_from_url)
    "1s7hGUCk6Aum4M9b3BhTn1LijGRWxSRF3",
]
INCLUDE_SUBFOLDERS = True
CSV_DIR = "data"  # where CSVs will be written
MASTER_CSV = os.path.join(CSV_DIR, "master_fasce_monthly_v2.csv")

SCOPES = ["https://www.googleapis.com/auth/drive.readonly"]

# ==============================
# Parsing helpers
# ==============================
MONTH_MAP = {
    "Gen": 1, "Feb": 2, "Mar": 3, "Apr": 4, "Mag": 5, "Giu": 6,
    "Lug": 7, "Ago": 8, "Set": 9, "Ott": 10, "Nov": 11, "Dic": 12,
}
ROW_START = re.compile(r'^(Gen|Feb|Mar|Apr|Mag|Giu|Lug|Ago|Set|Ott|Nov|Dic)-(\d{4})\b')
NUM_RE = re.compile(r'\d{1,3}(?:\.\d{3})*(?:,\d+)?|\d+(?:,\d+)?')
POD_RE = re.compile(r'\bPOD:\s*(IT[A-Z0-9]+)')  # e.g., "POD: IT012E00314418"

def itnum_to_float(s: str) -> float:
    """Convert Italian-formatted numbers to float."""
    s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except ValueError:
        return float("nan")

def extract_f1_f2_f3_from_line(line: str) -> Tuple[float, float, float] | None:
    """
    From a single 'Gen-2024 ...' row, return F1, F2, F3 (kWh).
    Assumes the first three numeric tokens after the month-year are F1, F2, F3.
    """
    line = re.sub(r'\s+', ' ', line.strip())
    line_wo_period = ROW_START.sub("", line, count=1).strip()
    nums = NUM_RE.findall(line_wo_period)
    vals = [itnum_to_float(n) for n in nums]
    if len(vals) < 3:
        return None
    return vals[0], vals[1], vals[2]

def parse_invoice_bytes_multi_pod(pdf_bytes: bytes, source_name: str) -> List[Dict]:
    """
    Parse an invoice (bytes) that may contain multiple POD sections.
    Returns a list of dict rows with columns:
      date, pod, source_file, days_in_month, F1_kWh, F2_kWh, F3_kWh, total_kWh, F1_daily_kWh, F2_daily_kWh, F3_daily_kWh
    """
    rows = []
    with pdfplumber.open(io.BytesIO(pdf_bytes)) as pdf:
        # we scan page-by-page to keep POD context changes tight
        current_pod = None
        in_consumo_storico = False

        for page in pdf.pages:
            text = page.extract_text() or ""
            lines = text.splitlines()

            for raw in lines:
                line = raw.strip()

                # Track POD when we see it (resets for each POD block)
                m_pod = POD_RE.search(line)
                if m_pod and m_pod != current_pod:
                    current_pod = m_pod.group(1)
                    # starting a new POD generally means we’re *not* inside a table yet
                    in_consumo_storico = False

                # Detect the start of the monthly table
                # Some PDFs have headers like "CONSUMO STORICO ENERGIA"
                if "CONSUMO STORICO ENERGIA" in line.upper():
                    in_consumo_storico = True
                    #print('yes!!!!')
                    # Next lines that match ROW_START are monthly data
                    continue

                # If we hit a different section header, stop reading table rows
                if in_consumo_storico and (
                    "SPESA ANNUA" in line.upper()
                    or "RIEPILOGO" in line.upper()
                    or "IMPORTO TOTALE DOCUMENTO" in line.upper()
                    or "PERIODO " in line
                ):
                    in_consumo_storico = False
                    #print('Why', line)
                
                # Parse monthly rows while we're inside the table
                if in_consumo_storico:
                    #print('I should be here with', line)
                    m = ROW_START.match(line)
                    if m:
                        mon_it, year = m.group(1), int(m.group(2))
                        month = MONTH_MAP.get(mon_it)
                        if not month:
                            continue
                        date = datetime(year, month, 1).date()
                        fvals = extract_f1_f2_f3_from_line(line)
                        if not fvals:
                            continue
                        F1_kWh, F2_kWh, F3_kWh = fvals
                        days_in_month = calendar.monthrange(year, month)[1]

                        rows.append({
                            "date": date.isoformat(),
                            "pod": current_pod or "UNKNOWN",
                            "source_file": source_name,
                            "days_in_month": days_in_month,
                            "F1_kWh": F1_kWh,
                            "F2_kWh": F2_kWh,
                            "F3_kWh": F3_kWh,
                            "total_kWh": F1_kWh + F2_kWh + F3_kWh,
                            "F1_daily_kWh": F1_kWh / days_in_month,
                            "F2_daily_kWh": F2_kWh / days_in_month,
                            "F3_daily_kWh": F3_kWh / days_in_month,
                        })
                    
    return rows

# ==============================
# Google Drive helpers
# ==============================
def folder_id_from_url(url_or_id: str) -> str:
    """Accept a Drive folder URL or an ID; return the ID."""
    m = re.search(r"/folders/([A-Za-z0-9_-]+)", url_or_id)
    return m.group(1) if m else url_or_id

def get_drive_service():
    """Authenticate with OAuth desktop flow (uses credentials.json, creates token.json)."""
    creds = None
    if os.path.exists("token.json"):
        try:
            creds = Credentials.from_authorized_user_file("token.json", SCOPES)
        except Exception:
            creds = None

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES)
            creds = flow.run_local_server(port=0)
        with open("token.json", "w") as token:
            token.write(creds.to_json())

    return build("drive", "v3", credentials=creds)

def list_folder_pdfs(service, folder_id: str, include_subfolders=True) -> List[Dict]:
    """
    Return [{id, name, mimeType}] for PDFs inside a folder (recurse into subfolders if requested).
    """
    results: List[Dict] = []

    def _list_in(fid):
        page_token = None
        while True:
            q = f"'{fid}' in parents and trashed=false"
            resp = service.files().list(
                q=q,
                fields="nextPageToken, files(id, name, mimeType)",
                includeItemsFromAllDrives=True,
                supportsAllDrives=True,
                pageToken=page_token
            ).execute()
            for f in resp.get("files", []):
                if f["mimeType"] == "application/pdf":
                    results.append(f)
                elif include_subfolders and f["mimeType"] == "application/vnd.google-apps.folder":
                    _list_in(f["id"])
            page_token = resp.get("nextPageToken")
            if not page_token:
                break

    _list_in(folder_id)
    return results

def download_file_bytes(service, file_id: str) -> bytes:
    """Download a Drive file (by id) into memory (bytes)."""
    request = service.files().get_media(fileId=file_id)
    fh = io.BytesIO()
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while not done:
        _, done = downloader.next_chunk()
    return fh.getvalue()

# ==============================
# Main
# ==============================
def main():
    os.makedirs(CSV_DIR, exist_ok=True)
    service = get_drive_service()

    # Gather PDFs from all configured folders
    all_files = []
    for fid in FOLDER_IDS:
        folder_id = folder_id_from_url(fid)
        all_files.extend(list_folder_pdfs(service, folder_id, INCLUDE_SUBFOLDERS))

    if not all_files:
        print("No PDFs found across the specified Drive folders.")
        return

    print(f"Found {len(all_files)} PDF(s). Parsing…")

    all_rows: List[Dict] = []
    for f in all_files:
        if f['name'].startswith('DET'):# or ('2024' not in f['name'] and '2023' not in f['name'] and  '2025' not in f['name']):
            continue
        try:
            print('Doing', f['name'])
            content = download_file_bytes(service, f["id"])
            rows = parse_invoice_bytes_multi_pod(content, f["name"])
            if rows:
                all_rows.extend(rows)
            else:
                print(f"[WARN] No monthly rows parsed in: {f['name']}")
        except Exception as e:
            print(f"[WARN] Failed to parse {f['name']}: {e}")

    df = pd.DataFrame(all_rows)
    if df.empty:
        print("⚠️ No 'Consumo Storico Energia' rows found.")
        return

    # Keep last by (pod, date, source_file) in case duplicates exist
    df.sort_values(["pod", "date", "source_file"], inplace=True)
    df = df.drop_duplicates(subset=["pod", "date"], keep="last")

    # Write master CSV
    df.to_csv(MASTER_CSV, index=False)
    print(f"✅ Master CSV -> {MASTER_CSV}")

    # Write one CSV per POD
    for pod, g in df.groupby("pod"):
        out_path = os.path.join(CSV_DIR, f"fasce_monthly_{pod}_full.csv")
        g.to_csv(out_path, index=False)
        print(f"✅ POD CSV -> {out_path}  (rows: {len(g)})")

    # Small preview
    print(df.head(12).to_string(index=False))

if __name__ == "__main__":
    main()


Found 125 PDF(s). Parsing…
Doing 1_E_0157095_20201229.pdf
Doing 1_E_0142928_20201130.pdf
Doing 1_E_0127356_20201029.pdf
Doing 1_E_0115336_20200930.pdf
Doing 1_E_0103231_20200828.pdf
Doing 1_E_0085415_20200728.pdf
Doing 1_E_0072131_20200617.pdf
Doing 1_E_0367614_20231224.pdf
Doing 1_E_0367613_20231224.pdf
Doing 1_E_0338956_20231128.pdf
Doing 1_E_0304301_20231030.pdf
Doing 1_E_0304300_20231030.pdf
Doing 1_E_0275549_20230927.pdf
Doing 1_E_0240690_20230827.pdf
Doing 1_E_0240689_20230827.pdf
Doing 1_E_0206529_20230728.pdf
Doing 1_E_0176305_20230629.pdf
Doing 1_E_0176304_20230629.pdf
Doing 1_E_0142843_20230528.pdf
Doing 1_E_0114078_20230426.pdf
Doing 1_E_0114077_20230426.pdf
Doing 1_E_0062606_20230322.pdf
Doing 1_E_0040429_20230226.pdf
Doing 1_E_0040428_20230226.pdf
Doing 1_E_0013951_20230130.pdf
Doing 1_E_0234648_20221229.pdf
Doing 1_E_0234647_20221229.pdf
Doing 1_E_0204817_20221130.pdf
Doing 1_E_0179316_20221028.pdf
Doing 1_E_0179315_20221028.pdf
Doing 1_E_0152743_20220929.pdf
Doing 1_E_01